In [1]:
!git clone https://github.com/Ro6ertWcislo/AutoESN.git

Cloning into 'AutoESN'...
remote: Enumerating objects: 598, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 598 (delta 14), reused 6 (delta 6), pack-reused 564
Receiving objects: 100% (598/598), 433.19 KiB | 10.07 MiB/s, done.
Resolving deltas: 100% (347/347), done.


In [2]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
from statsmodels.tsa.seasonal import seasonal_decompose
import optuna
import torch
from matplotlib import pyplot as plt
import numpy as np

import auto_esn.utils.dataset_loader as dl
from auto_esn.datasets.df import MackeyGlass
from auto_esn.esn.esn import GroupedDeepESN
from auto_esn.esn.reservoir.activation import self_normalizing_default
from auto_esn.esn.reservoir.util import NRMSELoss
from auto_esn.esn.reservoir.initialization import CompositeInitializer, WeightInitializer
from auto_esn.esn.reservoir.util import NRMSELoss

nrmse = NRMSELoss()

### Mackey Glass Dataset

four datasets - electricity, temperatur, [hungarian chickenpox](https://archive.ics.uci.edu/dataset/580/hungarian+chickenpox+cases), my own data



In [139]:
mg = pd.read_csv('mg.csv')

In [140]:
mg.head()

,y
0,1.200000
1,1.188060
2,1.176238
3,1.164535
4,1.152947


In [141]:
mg.columns

Index(['y'], dtype='object')

In [142]:
from sklearn.preprocessing import MinMaxScaler

def prepare_mg(mg):
    mg['date'] = pd.date_range(start="01-01-2000", periods=mg.shape[0], freq="D")
    mg.set_index('date', inplace = True)
    mg_train = mg[mg.index.year<2020]

    mg_val = mg[mg.index.year>=2020]


    columns_to_normalize = mg.columns
    scaler = MinMaxScaler()
    mg_train[columns_to_normalize] = scaler.fit_transform(mg_train[columns_to_normalize].values)
    mg_val[columns_to_normalize] = scaler.transform(mg_val[columns_to_normalize.values])
    return mg_train, mg_val, mg

In [143]:
mg_train, mg_val, mg = prepare_mg(mg)

<ipython-input-142-2051ded7c7df>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mg_train[columns_to_normalize] = scaler.fit_transform(mg_train[columns_to_normalize].values)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but MinMaxScaler was fitted without feature names
  warnings.warn(
<ipython-input-142-2051ded7c7df>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mg_val[columns_to_normalize] = scaler.transform(mg_val[columns_to_normalize.values])


In [144]:
mg_train.head()

,y
date,
2000-01-01,0.896060
2000-01-02,0.885151
2000-01-03,0.874351
2000-01-04,0.863658
2000-01-05,0.853071


In [145]:
def data_to_torch(data):
    a = np.array(data)
    return torch.from_numpy(a)

In [146]:
train_X, train_y = data_to_torch(mg_train['y'].values[:-1]), data_to_torch(mg_train['y'].values[1:])

In [147]:
test_X, test_y = data_to_torch(mg_val['y'].values[:-1]), data_to_torch(mg_val['y'].values[1:])

### esn definition

In [148]:
def best_deepesn_initializer(seed):
    # initialize input weights with uniform distribution from -1 to 1 and specified seed to reproduce results
    input_weight = CompositeInitializer().with_seed(seed).uniform()

    reservoir_weight = CompositeInitializer() \
        .with_seed(seed) \
        .uniform() \
        .sparse(density=0.1) \
        .spectral_normalize() \
        .scale(factor=1.0)  # unnecesary but i wanted to make it explicit

    return WeightInitializer(weight_ih_init=input_weight, weight_hh_init=reservoir_weight)

In [149]:
X, X_test, y, y_test = train_X, test_X, train_y, test_y

X = torch.reshape(X, (X.shape[0],1))
y = torch.reshape(y, (y.shape[0],1))

X_test = torch.reshape(X_test, (X_test.shape[0],1))
y_test = torch.reshape(y_test, (y_test.shape[0],1))

activation = self_normalizing_default(leaky_rate=1, spectral_radius=120)
seed = 70

### One step ahead prediction

In [150]:
def get_activation(leaky_rate = 1, radius = 120):
    return self_normalizing_default(leaky_rate=leaky_rate, spectral_radius=radius)

In [151]:
def get_esn(activation, input_size = 6, groups = 3, groups_def = (2,5,2), hidden_size =100, seed = 70):
    esn = GroupedDeepESN(
        input_size=input_size,
        groups=groups,
        num_layers=groups_def,
        hidden_size=hidden_size,
        initializer=best_deepesn_initializer(seed),
        activation=activation
    )
    return esn

### Optuna study

In [152]:
def objective(trial):
    activation = get_activation(
        leaky_rate = trial.suggest_float("leaky_rate", 0.85, 1.2),
        radius = trial.suggest_int("spectral radius", 100,800))
    esn = get_esn(activation, input_size = 1,hidden_size = trial.suggest_int("hidden_size",100,550))
    esn.fit(X, y)
    output = esn(X_test)
    return nrmse(output, y_test).item()

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2024-01-23 14:19:51,738] A new study created in memory with name: no-name-d0335ecd-997c-4bff-b702-3f7647776145
[I 2024-01-23 14:20:40,677] Trial 0 finished with value: 0.0018375540544008674 and parameters: {'leaky_rate': 1.107275706882336, 'spectral radius': 522, 'hidden_size': 266}. Best is trial 0 with value: 0.0018375540544008674.
[I 2024-01-23 14:21:38,195] Trial 1 finished with value: 0.00042038629571485405 and parameters: {'leaky_rate': 0.9845366984461589, 'spectral radius': 308, 'hidden_size': 307}. Best is trial 1 with value: 0.00042038629571485405.
[I 2024-01-23 14:24:21,610] Trial 2 finished with value: 0.0007472837800493897 and parameters: {'leaky_rate': 1.0642521910521512, 'spectral radius': 721, 'hidden_size': 522}. Best is trial 1 with value: 0.00042038629571485405.
[I 2024-01-23 14:24:43,593] Trial 3 finished with value: 0.0009692424631453368 and parameters: {'leaky_rate': 1.0559645510897322, 'spectral radius': 298, 'hidden_size': 154}. Best is trial 1 with value: 0.0

In [ ]:
study.best_params

#visualize one step ahead prediction

In [ ]:
esn = get_esn(get_activation(leaky_rate = study.best_params['leaky_rate'],
                             radius = study.best_params['spectral radius']),
              hidden_size = study.best_params['hidden_size'], input_size = 1)
esn.fit(X, y)

In [ ]:
X_test.shape, y_test.shape

In [ ]:
output = esn(X_test)
# evaluate
n = nrmse(output, y_test).item()
print('value of nrmse metric: ',n)
# plot
print(output.shape)
print(y_test.shape)

In [ ]:
plt.plot(range(len(X_test)), output.view(-1).detach().numpy(), 'r', label = 'results')
plt.plot(range(len(X_test)), y_test.view(-1).detach().numpy(), 'b', label = 'test y')
leg = plt.legend(loc='upper right')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.metrics import root_mean_squared_error
import math

print('values for one step ahead prediction')
print('nrmse : ',n)
print('mse : ', mean_squared_error(output, y_test))
print('mape : ', mean_absolute_percentage_error(output, y_test))
print('rmse : ',  math.sqrt(mean_squared_error(output, y_test)))

### Extrapolation multistep ahead prediction


In [ ]:
steps_ahead = 40
past_steps = 100

In [ ]:
train_y = []
train_X = []
for i in range(past_steps, len(mg_train)-steps_ahead):
    train_y.append(mg_train['y'].values[i+1:i+steps_ahead+1])
    train_X.append(mg_train['y'].values[i-past_steps:i])

In [ ]:
test_y = []
test_X = []
for i in range(past_steps, len(mg_val)-steps_ahead):
    test_y.append(mg_val['y'].values[i+1:i+steps_ahead+1])
    test_X.append(mg_val['y'].values[i-past_steps:i])

In [ ]:
train_y = data_to_torch(np.array(train_y))
train_X = data_to_torch(np.array(train_X))

In [ ]:
test_y = data_to_torch(np.array(test_y))
test_X = data_to_torch(np.array(test_X))

In [ ]:
train_X.shape, train_y.shape

In [ ]:
X, X_test, y, y_test = train_X, test_X, train_y, test_y
print(X.shape, X_test.shape)
X = torch.reshape(X, (X.shape[0],past_steps))
y = torch.reshape(y, (y.shape[0],steps_ahead))

X_test = torch.reshape(X_test, (X_test.shape[0],past_steps))
y_test = torch.reshape(y_test, (y_test.shape[0],steps_ahead))

activation = self_normalizing_default(leaky_rate=1, spectral_radius=120)
seed = 70

In [ ]:
def objective(trial):
    activation = get_activation(
        leaky_rate = trial.suggest_float("leaky_rate", 0.85, 1.2),
        radius = trial.suggest_int("spectral radius", 100,800))
    esn = get_esn(activation, input_size = 100,hidden_size = trial.suggest_int("hidden_size",100,550))
    esn.fit(X, y)
    output = esn(X_test)
    return nrmse(output, y_test).item()

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

In [ ]:
esn = get_esn(get_activation(leaky_rate = study.best_params['leaky_rate'],
                             radius = study.best_params['spectral radius']),
              hidden_size = study.best_params['hidden_size'], input_size = past_steps)

In [ ]:
esn.fit(X, y)

In [ ]:
output = esn(X_test)
# evaluate
n = nrmse(output, y_test).item()
print('value of nrmse metric: ',n)
print('mse : ', mean_squared_error(output, y_test))
print('mape : ', mean_absolute_percentage_error(output, y_test))
print('rmse : ',  math.sqrt(mean_squared_error(output, y_test)))
# plot
print(output.shape)
print(y_test.shape)

In [ ]:
plt.plot(range(steps_ahead), output[0], 'r', label = 'results')
plt.plot(range(steps_ahead), y_test[0], 'b', label = 'test y')
leg = plt.legend(loc='upper right')
plt.show()